In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st

## One-tailed t-test 

In a packing plant, a machine packs cartons with jars. It is supposed that a new machine will pack faster on average than the machine currently used. 

To test that hypothesis, the times each machine takes to pack ten cartons are recorded. 
The results are in seconds in the tables in the file files_for_lab/machine.txt. 

Assume that there is sufficient evidence to conduct the t-test, does the data provide sufficient evidence to show if one machine is better than the other?



In [2]:
machine_data = pd.read_csv('~/Desktop/data_analytics/Week_7/Day_2/Afternoon/lab-t-tests-p-values/files_for_lab/machine.txt', sep='\t', encoding='UTF16')
machine_data

,New machine,Old machine
0,42.1,42.7
1,41.0,43.6
2,41.3,43.8
3,41.8,43.3
4,42.4,42.5
5,42.8,43.5
6,43.2,43.1
7,42.3,41.7
8,41.8,44.0
9,42.7,44.1


Ho = avg(time(s)) for new machine ≤ avg(time(s)) for old machine
H1 = avg(time(s)) for new machine > avg(time(s)) for old machine

In [3]:
sample = machine_data['New machine']
pop_mean = machine_data['Old machine'].mean()
t_statistic, p_value = st.ttest_1samp(sample, pop_mean, alternative = "greater")
print("The t statistic of our sample is: {:.2f} and the corresponding p-value is: {:.2f}".format(t_statistic, p_value))

The t statistic of our sample is: -5.04 and the corresponding p-value is: 1.00


p-value = 1.00, 𝛼 = 0.05, p-value > 𝛼 therefore we accept H0. 

In conclusion: Since we cannot reject the null hypothesis, this means that new machine will pack faster on average than the machine currently used.

## Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon (file files_for_lab/pokemon.csv). 

Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. 

Our hypothesis is that the defense and attack scores are equal. 

Compare the two columns to see if there is a statistically significant difference between them and comment on your result.

In [4]:
pokemon = pd.read_csv('~/Desktop/data_analytics/Week_7/Day_2/Afternoon/lab-t-tests-p-values/files_for_lab/pokemon.csv')
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [5]:
pokemon.shape

(800, 13)

Ho = attack = defense scores

H1 = attack ≠ defense scores

Given the unequality on the alternative: "two-sided" test

In [6]:
t_statistic, p_value = st.ttest_rel(pokemon['Attack'], pokemon['Defense'])
print("The t statistic of our sample is: {:.2f} and the corresponding p-value is: {:.3f}".format(t_statistic, p_value))

The t statistic of our sample is: 4.33 and the corresponding p-value is: 0.000


p-value = 0.000, 𝛼 = 0.05/2, p-value < 𝛼 therefore we reject H0.

In conclusion: Since we can reject the null hypothesis, this means that there is a statistically significant difference between the columns "Attack" and "Defense".

## ANOVA 

Your task is to understand the problem and write down all the steps to set up ANOVA

#### Part I:

Suppose you are working as an analyst in a microprocessor chip manufacturing plant. You have been given the task of analyzing a plasma etching process with respect to changing Power (in Watts) of the plasma beam. 

Data was collected and provided to you to conduct statistical analysis and **check if changing the power of the plasma beam has any effect on the etching rate by the machine**. 

You will conduct ANOVA and check if there is **any difference in the mean etching rate for different levels of power**. 

- State the null hypothesis
- State the alternate hypothesis
- What is the significance level
- What are the degrees of freedom of the model, error terms, and total DoF


**State the null hypothesis.**

Ho: μ(etching rate)1 = μ(etching rate)2 ...

Ho = the mean etching rate for different levels of power is the same

**State the alternate hypothesis.**

H1: μ(etching rate)1 ≠ μ(etching rate)2 ...

H1 = the mean etching rate for different levels of power is different 

**What is the significance level?** 

95% confidence level therefore 𝛼 = 0.05

**What are the degrees of freedom of the model, error terms, and total DoF?**

Degrees of freedom of the model: 

dof1 = K - 1         (K being the number of groups)

dof2 = N - K         (N being the number of observations)

Error terms:

SST and SSE

Total DoF:

total_dof = dof1 + dof2


#### Part II:

In this section, use Python to conduct ANOVA.

What conclusions can you draw from the experiment and why?

In [7]:
plasma = pd.read_excel('~/Desktop/data_analytics/Week_7/Day_2/Afternoon/lab-t-tests-p-values/files_for_lab/anova_lab_data.xlsx')
plasma

,Power,Etching Rate
0,160 W,5.43
1,180 W,6.24
2,200 W,8.79
3,160 W,5.71
4,180 W,6.71
5,200 W,9.20
6,160 W,6.22
7,180 W,5.98
8,200 W,7.90
9,160 W,6.01


In [8]:
group_df = plasma.groupby('Power ')['Etching Rate'].agg(power_mean = 'mean', samples = 'size').reset_index()
group_df

,Power,power_mean,samples
0,160 W,5.792,5
1,180 W,6.238,5
2,200 W,8.318,5


In [9]:
# The variance of the group means from the global mean

S2T = 0
for power in plasma['Power '].unique():
    num_members_group = len(plasma[plasma['Power '] == power])
    S2T += num_members_group * ((plasma[plasma['Power '] == power]['Etching Rate'].mean() - plasma['Etching Rate'].mean()) ** 2)
S2T /= (plasma['Power '].nunique() - 1)

print("The value of S2T is {:.2f}".format(S2T))

The value of S2T is 9.09


In [10]:
# The variance of the values against the global mean

S2E = 0
for power in plasma['Power '].unique():
    for rate in plasma[plasma['Power '] == power]['Etching Rate']:
        S2E += (rate - plasma[plasma['Power '] == power]['Etching Rate'].mean()) ** 2
S2E /= ( len(plasma) - plasma['Power '].nunique())

print("The value of S2E is {:.2f}".format(S2E))

The value of S2E is 0.25


In [11]:
# F value (ratio of the variances)

F = S2T/S2E
print("The value of F is {:.2f}".format(F))

The value of F is 36.88


In [12]:
# Degrees of freedom value

dof1 = plasma['Power '].nunique() - 1
dof2 = len(plasma) - plasma['Power '].nunique()

print("Number of degrees of freedom d1:", dof1)
print("Number of degrees of freedom d2:", dof2)

Number of degrees of freedom d1: 2
Number of degrees of freedom d2: 12


In [13]:
# The probability to get any F value lower or equal to F can be obtained with the CDF

st.f.cdf(F, dfn = dof1, dfd = dof2)

0.9999924934157276

In [14]:
# The probability to get a value bigger than F

1 - st.f.cdf(F, dfn = dof1, dfd = dof2)

7.5065842723986975e-06

In [15]:
# The critical value which corresponds to an area of 0.05

f_crit = st.f.ppf(1-0.05, dfn = dof1, dfd = dof2)

print("The critical value which corresponds to an area of 0.05 is: {:.2f}".format(f_crit))

The critical value which corresponds to an area of 0.05 is: 3.89


**In summary:**

S2T = 9.09

S2E = 0.25 - then use both to calculate F:

F = 36.88

Number of degrees of freedom d1: 2

Number of degrees of freedom d2: 12

probability to get value ≤ F: 0.999

probability to get value > F: 7.5e-06 

The critical value which corresponds to an area of 0.05 is: 3.89

As our F is bigger than the critical value, F > f_crit (36.88>3.89) **we reject H0**

value > F: 7.5e-06 = 7.5e-06 < 0.05 **we reject H0**

The Ho was: Ho = the mean etching rate for different levels of power is the same

Since we can reject it, it means that changing the power of the plasma beam has an effect on the etching rate by the machine


In [16]:
# doing by the spicy function

t_statistic, p_value = st.f_oneway(plasma[plasma['Power '] == '160 W']['Etching Rate'], plasma[plasma['Power '] == '180 W']['Etching Rate'], 
plasma[plasma['Power '] == '200 W']['Etching Rate'])

print("The t statistic of our sample is: {:.2f} and the corresponding p-value is: {:.3f}".format(t_statistic, p_value))

The t statistic of our sample is: 36.88 and the corresponding p-value is: 0.000


The above match the previous findings
p_value is 0.000 < 0.05 -> **we reject the H0**